In [1]:
def column(m, c):
  return [m[i][c] for i in range(len(m))]

def row(m, r):
  return m[r][:]
  
def height(m):
  return len(m)
  
def width(m):
  return len(m[0])

 
def gauss_seidel(m, x0=None, eps=1e-6, round=1000):
  n  = height(m)
  b  = column(m, n)
  x0 = [0] * n if x0 == None else x0
  x1 = x0[:]                 # change
 
  for k in range(round):
    for i in range(n):
      s = sum(-m[i][j] * x1[j] for j in range(n) if i != j)  
      x1[i] = (s + b[i]) / m[i][i]
    if all(abs(x1[i]-x0[i]) < eps for i in range(n)):
        return x1 
    x0 = x1[:]             
  raise ValueError('Solution does not converge')


def gaussian_elimination(m):
    
  # forward elimination
  n = height(m)
  for i in range(n):
    for j in range(i+1, n):
      v = [m[j][k] - (m[i][k] * m[j][i] / m[i][i]) for k in range(n+1)]
      m[j] = v
   
  if m[n-1][n-1] == 0: raise ValueError('No unique solution')
   
  # backward substitution
  x = [0] * n
  x[n-1] = m[n-1][n] / m[n-1][n-1]
  for i in range(n-2, -1, -1):
    s = sum(m[i][j] * x[j] for j in range(i, n))
    x[i] = (m[i][n] - s) / m[i][i]
  return x

In [2]:
def newton_nonlinear(f, j, x0, sys_lin_method=gauss_seidel, eps=1e-6, max_iteration=100):
    
  """
  Parameters
  ----------
  f  : list of functions
  j  : list of list of functions : Jacobian matrix
  x0 : list of floats            : initial guess
  sys_lin_meth : function : solve system of linear equations
  
  Returns
  -------  
  list of floats
  
  Raises
  ------
  ValueError : if solution doesn't converge
  """

  for n in range(max_iteration):
    # solve for dx
    m = [[fn(x0) for fn in j[i]] for i in range(len(j))]
    for i in range(len(f)):
      m[i].append(-f[i](x0))
    dx = sys_lin_method(m)
    # update x0
    x0 = [x0[i] + dx[i] for i in range(len(dx))]
    # terminate?
    if all(abs(i) < eps for i in dx): 
      return x0
  raise ValueError('Solution does not converge')

if __name__ == '__main__':
  
  f = [lambda x : (x[0]**2)/(186**2) + (x[1]**2)/(300**2 - 186**2) - 1, 
       lambda x : ((x[1]-500)**2)/(279**2) - ((x[0]-300)**2)/(500**2 - 279**2) - 1]
  j = [[lambda x :  2*x[0]/(186**2), lambda x : 2*x[1]/(300**2 - 186**2)],
       [lambda x : -2*(x[0]-300)/(500**2 - 279**2) , lambda x : 2*(x[1]-500)/(279**2)]]
  
  # initial guess
  x0 = [200, 200]
  
  # solve by Newton method
  x = newton_nonlinear(f, j, x0)

  print('solution is ', x)
  for i in range(len(f)):
    print('f[%d]=%f' % (i, f[i](x)))


solution is  [107.20337484756455, 192.35164605733192]
f[0]=0.000000
f[1]=0.000000
